<a href="https://colab.research.google.com/github/lahirukavinda/ADL-AI-Summit---Pre-Hackathon/blob/main/ADLHackothon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [118]:
#from google.colab import files
#uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
df = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset.csv', error_bad_lines=False).fillna(0)
df.shape[0]

10500

In [120]:
#df = df.astype('category')
df.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0,PKG2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,PKG6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0,PKG4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0,PKG4


In [121]:
x = df.drop('next_month_plan', axis='columns')
y = df['next_month_plan'].str.replace("PKG",'')
y.head()

0    2
1    1
2    6
3    4
4    4
Name: next_month_plan, dtype: object

In [122]:
from sklearn.preprocessing import LabelEncoder

#y_n = LabelEncoder().fit_transform(y)
y_n = y

features = ["device_type", "device_category", "gender", "district_name", "age_group"]
x["device_type"] = LabelEncoder().fit_transform(x["device_type"].astype(str))
x["device_category"] = LabelEncoder().fit_transform(x["device_category"].astype(str))
x["gender"] = LabelEncoder().fit_transform(x["gender"].astype(str))
x["district_name"] = LabelEncoder().fit_transform(x["district_name"].astype(str))
x["age_group"] = LabelEncoder().fit_transform(x["age_group"].astype(str))

#x = x.apply(LabelEncoder().fit_transform)

x.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,6875768,3,5,2,11,3,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0
1,6406277,3,5,2,5,3,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0
2,3563570,3,5,2,10,2,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0
3,1853666,3,5,1,4,3,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0
4,2794331,3,5,1,10,4,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0


In [123]:
x_train, x_test, y_train, y_test = train_test_split(x, y_n, random_state = 0) 

In [124]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# #model_mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(1000,8))
# model_mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(100,8)) #0.37
# #model_mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(8,8)) #0.36
# #model_mlp = MLPClassifier(max_iter=500, activation='relu') #hidden_layer_sizes=(3,2) #0.32
# print('Model MLP:\n', model_mlp)
# model_mlp.fit(x_train, y_train)
# print('Prediction Score of MLP: ', model_mlp.score(x_test, y_test))
# mlp_predictions = model_mlp.predict(x_test)
# cm_mlp = confusion_matrix(y_test, mlp_predictions)
# print('Confusion matrix of MLP:\n', cm_mlp)
# print('Clasification Report of MLP:\n',classification_report(y_test, mlp_predictions))

In [125]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

model_xg = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=30,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

# model_xg = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
#        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
#        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1) #0.38

model_xg.fit(x_train, y_train)
print('Prediction Score : ', model_xg.score(x_test, y_test))
xg_predictions = model_xg.predict(x_test)
cm_xg = confusion_matrix(y_test, xg_predictions)
print('Clasification Report :\n', classification_report(y_test, xg_predictions))

Prediction Score :  0.5436190476190477
Clasification Report :
               precision    recall  f1-score   support

           1       0.68      0.80      0.74       937
           2       0.28      0.11      0.16       342
           3       0.42      0.60      0.50       398
           4       0.27      0.12      0.17       187
           5       0.60      0.73      0.66       381
           6       0.39      0.31      0.34       203
           7       0.28      0.19      0.23        89
           8       0.38      0.32      0.35        88

    accuracy                           0.54      2625
   macro avg       0.41      0.40      0.39      2625
weighted avg       0.50      0.54      0.51      2625



In [126]:
df_test = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/test_dataset.csv', error_bad_lines=False).fillna(0)

df_test["device_type"] = LabelEncoder().fit_transform(df_test["device_type"].astype(str))
df_test["device_category"] = LabelEncoder().fit_transform(df_test["device_category"].astype(str))
df_test["gender"] = LabelEncoder().fit_transform(df_test["gender"].astype(str))
df_test["district_name"] = LabelEncoder().fit_transform(df_test["district_name"].astype(str))
df_test["age_group"] = LabelEncoder().fit_transform(df_test["age_group"].astype(str))

#df_test = df_test.apply(LabelEncoder().fit_transform)

df_test.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,2003793,3,6,2,4,3,4,689.42,148.0,0.000000,148.146634,37.037382,4.0,38.251243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0
1,1776101,3,6,2,4,3,7,799.93,300.0,0.180222,119.466825,10.027957,30.0,24.125172,0.0,0.0,0.0,0.0,0.0,0.0,107.0,16.266667,0.0,21.0,5.107143,4.924111,0.0,0.0,0.0,0.0
2,6945050,1,1,2,4,1,20,1538.45,8215.0,0.000000,4048.511104,2053.950266,4.0,932.358667,0.0,0.0,0.0,0.0,0.0,0.0,108.0,90.000000,0.0,8.0,13.558333,16.397071,0.0,0.0,110.0,2.0
3,2472049,3,6,1,16,3,10,995.75,11613.0,0.000000,1057.448432,430.118721,27.0,268.457304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416667,0.0,2.0,0.333333,0.087355,0.0,0.0,0.0,0.0
4,7197266,3,6,2,10,2,154,553.61,26505.0,28.515345,2490.832742,883.505624,30.0,660.353290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,350.0,1.0


In [127]:
xg_predictions_test = model_xg.predict(df_test)

In [128]:
df_test['next_month_plan'] = xg_predictions_test

In [129]:
df_test = df_test[['primary_identifier','next_month_plan']]
df_test.head()

,primary_identifier,next_month_plan
0,2003793,2
1,1776101,1
2,6945050,3
3,2472049,1
4,7197266,1


In [130]:
df_test.to_csv('/content/drive/MyDrive/Projects/ADLHackothon/submission.csv', mode = 'w', index=False)